# Report

In this notebook we report progress on the project of house price prediction.

In [1]:
import os
from functools import partial

In [52]:
import joblib
import pandas as pd

In [12]:
import app
import data
import model
import metrics

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
models_dir = "models"

In [6]:
dataset_path = "dataset.csv"

In [7]:
dataset = data.get_dataset(
    partial(pd.read_csv, filepath_or_buffer=dataset_path),
    splits=("train", "test")
)

**If you need to visualize anything from your training data, do it here**

## Baseline

Before doing any complex Machine Learning model, let's try to solve the problem by having an initial educated guess. 

In [98]:
model_path = os.path.join("models", "2021-05-29 17:22:00+00:00", "model.joblib")
reg = joblib.load(model_path)

In [100]:
train_predictions = reg.predict(dataset["train"][0])

In [103]:
test_predictions = reg.predict(dataset["test"][0])

In [105]:
train_error = metrics.custom_error(dataset["train"][1], train_predictions)
test_error = metrics.custom_error(dataset["test"][1], test_predictions)
print(f"Train error: {train_error}, Test error: {test_error}")

Train error: 33609.9990756996, Test error: 34799.09487677742


## Linear Regression Model 

We want to try easy things first, so know lets see how a linear regression model does.

In [ ]:
model_path = 
model =
train_predictions = 
test_predictions = 
train_error =
test_error = 

## Linear regression with Feature Engineering

Probably the previous model is not good enough, let's see how is the performance of a model using some produced features.  

## Regularized Linear Regression

Let's assume you are overfitting. Load the results of a linear regression model with regularized loss

In [ ]:
model_path = 
model =
train_predictions = 
test_predictions = 
train_error =
test_error = 

## Decision Tree

Decision trees ofer great complexity, they can fit even a noisy dataset almost perfectly. Let's see how it behaves on the task at hand. 

**Overfiting case**
Let's see the results for a model that has greatly overfit the data, this wouldn't be an ideal model, but at least it could tell that our model is powerful enough for the task at hand

In [ ]:
model_path = 
model =
train_predictions = 
test_predictions = 
train_error =
test_error = 

**Using best hyper params** Now let's see thow much a simple decision tree can give us

In [ ]:
model_path = 
model =
train_predictions = 
test_predictions = 
train_error =
test_error = 

## Random Forest

Now it is time to use a model that can properly help us to regularize the previous one.

In [ ]:
model_path = 
model =
train_predictions = 
test_predictions = 
train_error =
test_error = 